In [1]:
# Import dependencies 
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sqlalchemy import create_engine
from sqlalchemy import create_engine




In [2]:
# Load data from Resources as tab seperated file into dataframe
pd.set_option('display.max_columns', None)
mental_df = pd.read_csv('cleaned_data.csv')
mental_df

,Q1-Volunteered For Arts/Culture/Heritage,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,Q3-Interested in Volunteering,Q3-Interested in Others,Are you stressed about adjustment in new environment,Are you worried about academic issues,Are you stressed about financial issues,Are you stressed about your family,Are you anxious about friendships,Are you happy in your romantic relationship,Are you stressed about your health and wellbeing,Are you stressed for career,"Do you involve in clubs,activities and other events in college",Stressed about others,Career,Year of Study,Primary Programme,Gender,Housing Type,Q2-How many events have you Participated in ?,Q3-How many activities are you Interested in ?,Q5-What are your levels of stress ?,Q6-How Satisfied You are with your Student Life ?,Q7-How much effort do you make to interact with others ?,Q8-About How events are you aware about ?,Q9-What is an ideal student life ?
0,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,UGRD,2,Bachelor of Science,Male,Halls,1,1,2,2,2.0,2.0,just want everything to go smooth. serious
1,No,No,Yes,Yes,No,No,Yes,Yes,No,No,Yes,Yes,No,Yes,No,No,No,Yes,No,UGRD,3,Bachelor of Engineering,Female,Out of Campus,3,4,4,2,2.0,2.0,a mixture of both academic and non-academic
2,No,Yes,Yes,No,Yes,No,No,No,No,No,Yes,No,No,Yes,No,No,No,No,No,UGRD,1,Bachelor of Science,Female,Residential Colleges,2,3,2,3,3.0,1.0,"One where I�m doing well academically, involve..."
3,No,Yes,Yes,Yes,No,No,No,No,No,Yes,Yes,Yes,No,No,No,No,Yes,Yes,No,UGRD,2,Bachelor of Science,Female,Halls,1,3,5,2,2.0,1.0,"Good company around you, good grades, involvem..."
4,No,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,No,No,Yes,Yes,No,UGRD,2,Bachelor of Science,Male,Halls,2,4,4,2,3.0,1.0,Doing what I like and being happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,No,Yes,Yes,No,Yes,Yes,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,UGRD,1,Bachelor of Science,Female,Residential Colleges,1,4,2,2,2.0,3.0,work life balance
1352,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,No,No,No,No,No,No,No,No,UGRD,3,Bachelor of Science,Male,Halls,3,5,1,2,2.0,2.0,fun and engaging and experiential and learn
1353,No,No,Yes,Yes,Yes,No,No,Yes,No,Yes,Yes,No,No,Yes,No,No,Yes,Yes,No,UGRD,3,Bachelor of Engineering,Male,Halls,3,4,5,2,2.0,2.0,It enriches and makea your university life com...
1354,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,Yes,No,No,UGRD,1,Bachelor of Science (SDE),Male,Halls,2,1,1,2,2.0,2.0,a balance between study and fun


In [3]:
# build a dataframe of the questions
dict = {}
count = 0
for column in mental_df.columns:
    
    dict[count] = [column]
    count = count + 1
    
    
dict

questions_df = pd.DataFrame.from_dict(dict, orient='index', columns=['Question'])
questions_df

,Question
0,Q1-Volunteered For Arts/Culture/Heritage
1,Q3-Interested in Arts & Culture
2,Q3-Interested in Science & Technology
3,Q3-Interested in Research and independent study
4,Q3-Interested in Sports
5,"Q3-Interested in Other competitions (eg case, ..."
6,Q3-Interested in Entrepreneurship
7,Q3-Interested in Volunteering
8,Q3-Interested in Others
9,Are you stressed about adjustment in new envir...


In [8]:
# rename the columns in the dataframe to numbers
column_rename = {}
count = 0
for column in mental_df.columns:
    
    column_rename[column] = count
    count = count + 1
    
    
column_rename

renamed_df = mental_df.rename(columns=column_rename)
renamed_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,UGRD,2,Bachelor of Science,Male,Halls,1,1,2,2,2.0,2.0,just want everything to go smooth. serious
1,No,No,Yes,Yes,No,No,Yes,Yes,No,No,Yes,Yes,No,Yes,No,No,No,Yes,No,UGRD,3,Bachelor of Engineering,Female,Out of Campus,3,4,4,2,2.0,2.0,a mixture of both academic and non-academic
2,No,Yes,Yes,No,Yes,No,No,No,No,No,Yes,No,No,Yes,No,No,No,No,No,UGRD,1,Bachelor of Science,Female,Residential Colleges,2,3,2,3,3.0,1.0,"One where I�m doing well academically, involve..."
3,No,Yes,Yes,Yes,No,No,No,No,No,Yes,Yes,Yes,No,No,No,No,Yes,Yes,No,UGRD,2,Bachelor of Science,Female,Halls,1,3,5,2,2.0,1.0,"Good company around you, good grades, involvem..."
4,No,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,No,No,Yes,Yes,No,UGRD,2,Bachelor of Science,Male,Halls,2,4,4,2,3.0,1.0,Doing what I like and being happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,No,Yes,Yes,No,Yes,Yes,No,No,No,No,Yes,No,No,No,No,No,Yes,No,No,UGRD,1,Bachelor of Science,Female,Residential Colleges,1,4,2,2,2.0,3.0,work life balance
1352,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,No,No,No,No,No,No,No,No,UGRD,3,Bachelor of Science,Male,Halls,3,5,1,2,2.0,2.0,fun and engaging and experiential and learn
1353,No,No,Yes,Yes,Yes,No,No,Yes,No,Yes,Yes,No,No,Yes,No,No,Yes,Yes,No,UGRD,3,Bachelor of Engineering,Male,Halls,3,4,5,2,2.0,2.0,It enriches and makea your university life com...
1354,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,Yes,No,No,UGRD,1,Bachelor of Science (SDE),Male,Halls,2,1,1,2,2.0,2.0,a balance between study and fun


In [9]:
# Create an in-memory SQLite database

# Create the engine
engine = create_engine("sqlite:///mental_health.db", echo=False)

# Reflect existing db into a new model
Base = automap_base()
# Refelct the tables
Base.prepare(engine, reflect=True)

In [10]:
# Use pandas to_sql to write records stored in a dataframe to a SQL database

# Write dataframe of all survey questions to a sqlite table
renamed_df.to_sql('pre_encoded_survey', con=engine, if_exists='replace')
# Write dataframe of all questions with encoded numbers to a sqlite table
questions_df.to_sql('pre_encoded_questions', con=engine, if_exists='replace')

31

In [7]:
# Check that you can select a table from the database
#engine.execute("SELECT * FROM pre_encoded_survey").fetchall()
engine.execute("SELECT * FROM pre_encoded_questions").fetchall()

[(0, 'Does your employer provide mental health benefits as part of healthcare coverage?'),
 (1, 'Do you know the options for mental health care available under your employer-provided coverage?'),
 (2, 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?'),
 (3, 'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:'),
 (4, 'Do you think that discussing a mental health disorder with your employer would have negative consequences?'),
 (5, 'Do you think that discussing a physical health issue with your employer would have negative consequences?'),
 (6, 'Do you feel that your employer takes mental health as seriously as physical health?'),
 (7, 'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?'),
 (8, 'Do you have previous employers?'),
 (9, 'Have your previous employers provided mental health benefi